In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D, Flatten, MaxPooling1D, PReLU
from keras.initializers import Constant

In [6]:
def evaluate_model(train_x, train_y, test_x, test_y, dropout, epochs, batch_size):
    model = Sequential()

    model.add(Conv1D(filters=64, kernel_size=5, input_shape=(200, 1), padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(PReLU(alpha_initializer=Constant(value=0.25)))
    model.add(Dropout(dropout))

    model.add(Conv1D(filters=128, kernel_size=5, padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(PReLU(alpha_initializer=Constant(value=0.25)))
    model.add(Dropout(dropout))

    model.add(Conv1D(filters=128, kernel_size=5, padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(PReLU(alpha_initializer=Constant(value=0.25)))
    model.add(Dropout(dropout))

    model.add(Conv1D(filters=128, kernel_size=3, padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(PReLU(alpha_initializer=Constant(value=0.25)))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(30))
    model.add(PReLU(alpha_initializer=Constant(value=0.25)))
    model.add(Dense(2, activation='softmax'))


    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=1)
    # evaluate model
    loss, accuracy = model.evaluate(test_x, test_y, batch_size=batch_size, verbose=0)
    return history, accuracy, loss

In [12]:
df = pd.read_csv('../data/arrhythmia_sliced_30.csv')
X = df.iloc[:, :-1]
y = df['class']
y = to_categorical(y)

X = np.expand_dims(X, axis=2)

train_x, test_x, train_y, test_y = \
                    train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
train_history, test_accuracy, test_loss = evaluate_model(train_x, train_y, test_x, test_y, 0.5, 50, 32)

Epoch 1/50
761/761 [==============================] - 4s 5ms/step - loss: 1.2642 - accuracy: 0.6334
Epoch 2/50
761/761 [==============================] - 2s 3ms/step - loss: 0.7295 - accuracy: 0.6873
Epoch 3/50
761/761 [==============================] - 2s 3ms/step - loss: 0.6262 - accuracy: 0.7227
Epoch 4/50
761/761 [==============================] - 2s 3ms/step - loss: 0.5646 - accuracy: 0.7201
Epoch 5/50
761/761 [==============================] - 3s 3ms/step - loss: 0.5483 - accuracy: 0.7385
Epoch 6/50
761/761 [==============================] - 2s 3ms/step - loss: 0.5484 - accuracy: 0.7346
Epoch 7/50
761/761 [==============================] - 3s 4ms/step - loss: 0.5487 - accuracy: 0.7227
Epoch 8/50
761/761 [==============================] - 2s 3ms/step - loss: 0.5421 - accuracy: 0.7267
Epoch 9/50
761/761 [==============================] - 3s 4ms/step - loss: 0.5442 - accuracy: 0.7319
Epoch 10/50
761/761 [==============================] - 2s 3ms/step - loss: 0.5356 - accuracy: 0.7411

In [13]:
def visualize_history(history):
    plt.plot(history.history['accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train accuracy', 'train loss'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [14]:
test_accuracy

0.8219895362854004

In [16]:
from numpy import loadtxt
from keras.models import load_model
# load model
model = load_model('../models/model.h5')
# summarize model.
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 200, 64)           384       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 100, 64)           0         
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 100, 64)           6400      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 100, 128)          41088     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 50, 128)           0         
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 50, 128)          